In [1]:
using DataFrames, LinearAlgebra, Distributions, Revise, Serialization

In [2]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

┌ Info: Precompiling BLPmodule [top-level]
└ @ Base loading.jl:1317


---
#### Mean utilities

In [3]:
df = deserialize("jls/df.jls");
firm_IDs_long = df.j;
tract_IDs_long = df.t;
X = Matrix{Float64}(df[!, [:x1, :x2]])
D = Matrix{Float64}(df[!, [:d, :d2]])
Q = df.q
M = df.M;
nJ = length(unique(firm_IDs_long));
nI = 100;

In [19]:
pars = m.set_Pars(σ = ones(2), K = 2, nI = nI, δs=ones(nJ,1));
serialize("jls/pars.jls", pars)

---

In [20]:
ec = m.make_Economy(
    firm_IDs_long,
    tract_IDs_long,
    X,
    D,
    Q,
    M,
    nI
    );


Economy with 394 firms and 3110 tracts.


In [21]:
serialize("jls/ec.jls", ec)

---

In [7]:
Threads.nthreads()

16

In [22]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
deltas, q_iter = m.compute_deltas(ec, pars,max_iter=1000, verbose=true);
# serialize("jls/deltas.jls", deltas);

(K, nI, σ, δs[1:5]) = (2, 100, [1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0])
(in /mnt/staff/zhli/blp-quantities/BLPmodule/solve.jl:43)
initial_δ[1:5] = [1.0, 1.0, 1.0, 1.0, 1.0]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/solve.jl:54)
q_obs[1:5] = [80.0, 81.0, 54.0, 55.0, 121.0]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/solve.jl:62)
q_iter[1:5] = [1063.808935313926, 1718.4110486610832, 748.4408510986774, 639.2442853327319, 1056.3460085453146]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/solve.jl:63)
δs[1:5] = [-1.587584447029335, -2.0547061792426446, -1.6290081309246474, -1.4529534892551146, -1.1667805246080851]
δ_[1:5] = [-1.587584447029335, -2.0547061792426446, -1.6290081309246474, -1.4529534892551146, -1.1667805246080851]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/solve.jl:65)
dist = 0.0, iterations = 1


In [10]:
using BenchmarkTools

In [11]:
ec = deserialize("jls/ec.jls");
it = 1
@benchmark m.compute_deltas(ec, pars, max_iter=it, verbose=false)

BenchmarkTools.Trial: 262 samples with 1 evaluation.
 Range (min … max):  12.437 ms … 382.207 ms  ┊ GC (min … max):  0.00% … 96.25%
 Time  (median):     12.817 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   20.145 ms ±  48.258 ms  ┊ GC (mean ± σ):  34.86% ± 14.19%

  █                                                             
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▄ ▅
  12.4 ms       Histogram: log(frequency) by time       348 ms <

 Memory estimate: 27.93 MiB, allocs estimate: 78287.

In [ ]:
ec = deserialize("jls/ec.jls");
it = 10
@benchmark m.compute_deltas(ec, pars,max_iter=it, verbose=false)

BenchmarkTools.Trial: 91 samples with 1 evaluation.
 Range (min … max):  35.789 ms … 432.906 ms  ┊ GC (min … max):  0.00% … 89.81%
 Time  (median):     42.641 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   55.200 ms ±  53.204 ms  ┊ GC (mean ± σ):  22.56% ± 19.73%

   ██▁                                                          
  ▆███▅▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▃▃▃ ▁
  35.8 ms         Histogram: frequency by time          206 ms <

 Memory estimate: 76.84 MiB, allocs estimate: 415771.

In [12]:
ec = deserialize("jls/ec.jls");
it = 1000
@benchmark m.compute_deltas(ec, pars,max_iter=it, verbose=false)

BenchmarkTools.Trial: 5 samples with 1 evaluation.
 Range (min … max):  827.358 ms …    1.528 s  ┊ GC (min … max): 15.49% … 52.50%
 Time  (median):     964.711 ms               ┊ GC (median):    28.05%
 Time  (mean ± σ):      1.111 s ± 332.949 ms  ┊ GC (mean ± σ):  36.12% ± 17.26%

  █          ▁                                      ▁         ▁  
  █▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█ ▁
  827 ms           Histogram: frequency by time          1.53 s <

 Memory estimate: 1.50 GiB, allocs estimate: 10503294.

In [ ]:
# # this was the line that the memory profiler said was creating the vast majority of allocations
# # ...which doesn't make sense because it's not even in the inner loop
# # ...and it's the inner loop that's driving the allocations and times (according to benchmarks)
# @time for t in ec.tracts
#     t.abδ .= t.D * pars.nlcoefs
# end

___
continued in `gmm.ipynb`